In [ ]:
import re
import os
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

# Example!!

# Load a sentence embedding model
model = SentenceTransformer('all-mpnet-base-v2')

# Define L1 and L2 categories
l1_categories = ["Food and Beverages", "Healthcare", "Manufacturing"]
l2_categories = ["Thai Restaurant", "Clinic", "Manufacture of Electronics"]

# Compute embeddings
l1_embeddings = model.encode(l1_categories, convert_to_tensor=True)
l2_embeddings = model.encode(l2_categories, convert_to_tensor=True)

# Create a DataFrame for L2 categories
l2_df = pd.DataFrame({'INDUSTRY': l2_categories})

# Add columns for predicted L1 categories and confidence
l2_df['PREDICTED_L1'] = ""
l2_df['CONFIDENCE'] = 0.0

# Compute cosine similarity and assign predicted L1 and confidence
for i, l2 in enumerate(l2_categories):
    similarities = util.cos_sim(l2_embeddings[i], l1_embeddings)
    best_match_idx = similarities.argmax().item()
    l2_df.loc[i, 'PREDICTED_L1'] = l1_categories[best_match_idx]
    l2_df.loc[i, 'CONFIDENCE'] = similarities[0, best_match_idx].item() 

# Check resulting DF
l2_df.head()